## HW2_Q11_Chami

### part a: MLE estimate numerical

In [2]:
import numpy as np
from scipy import optimize, stats

In [3]:
with open("normal.data.txt") as f: 
    file = f.read()

values = [float(x) for x in file.split()]
data = np.array(values) 

# closed form solution and numbers 
n = len(data) 
mu_hat = sum(data) / n                          # mean
sigma2_hat = ((data - mu_hat)**2).sum() / n     # variance

def neg_log_like(theta, data): 
    mu, sigma2 = theta
    
    if sigma2 <= 0: 
        return np.inf
        
    out = 0.5 * n * np.log(2 * np.pi * sigma2) + np.sum((data - mu)**2) / (2 * sigma2)
    return out


In [4]:
theta0 = [0, 0]
result = optimize.minimize(neg_log_like, theta0, args=(data,), bounds=[(None,None), (1e-8,None)])
mu_hat_num, sigma2_hat_num = result.x

print("Numerical MLE:\n")
print(f"mu_hat numerical={mu_hat_num}", f", sigma2_hat numerical={sigma2_hat_num}")
print("Closed-form MLE:\n")
print(f"mu_hat closed form={mu_hat}", f", sigma2_hat closed form={sigma2_hat}" )

Numerical MLE:

mu_hat numerical=100.48051368314435 , sigma2_hat numerical=216.31867036591245
Closed-form MLE:

mu_hat closed form=100.48049999999999 , sigma2_hat closed form=216.31846975


### part b: Showing that the Minus Log-Likelihood Is Minimized

In [5]:
nll_hat = neg_log_like([mu_hat, sigma2_hat], data)

# small perturbations
eps_mu = 0.01
eps_s2 = 0.5

nll_mu_up = neg_log_like([mu_hat + eps_mu, sigma2_hat], data)
nll_mu_down = neg_log_like([mu_hat - eps_mu, sigma2_hat], data)

nll_s2_up = neg_log_like([mu_hat, sigma2_hat + eps_s2], data)
nll_s2_down = neg_log_like([mu_hat, sigma2_hat - eps_s2], data)

print("NLL at MLE:", nll_hat)
print("NLL mu + eps:", nll_mu_up)
print("NLL mu - eps:", nll_mu_down)
print("NLL sigma2 + eps:", nll_s2_up)
print("NLL sigma2 - eps:", nll_s2_down)


NLL at MLE: 821.4628785228199
NLL mu + eps: 821.4629247509575
NLL mu - eps: 821.4629247509575
NLL sigma2 + eps: 821.4631448317826
NLL sigma2 - eps: 821.4631464783167


### part c: Estimated Asymptotic Covariance Matrix of the MLEs

In [8]:
# estimated asymptotic covariance matrix
asymptotic_var_hat = np.array([
    [sigma2_hat / n, 0.0],
    [0.0, 2 * sigma2_hat**2 / n]
])

print(asymptotic_var_hat)


[[  1.08159235   0.        ]
 [  0.         467.93680355]]


### part d: better Asymptotic Covariance Matrix of the MLEs

In [10]:
s2_unbiased = np.var(data, ddof=1)

# better estimated asymptotic covariance matrix
avar_better = np.array([
    [s2_unbiased / n, 0.0],
    [0.0, 2 * s2_unbiased**2 / n]
])

print("Better estimated asymptotic covariance matrix:\n", avar_better)


Better estimated asymptotic covariance matrix:
 [[  1.08702749   0.        ]
 [  0.         472.65150229]]


### part e: confidence interval 

In [18]:
se_sigma2 = np.sqrt(2 * sigma2_hat**2 / n)

ci_sigma2 = (
    sigma2_hat - 1.96 * se_sigma2,
    sigma2_hat + 1.96 * se_sigma2
)

print("95% large-sample CI for sigma^2:", ci_sigma2)


95% large-sample CI for sigma^2: (np.float64(173.920049679), np.float64(258.716889821))


### part f: hypothesis testing

In [11]:
mu0 = 103

z_stat = (mu_hat - mu0) / np.sqrt(sigma2_hat / n)
p_value_z = 2 * (1 - stats.norm.cdf(abs(z_stat)))

print("Z-test statistic:", z_stat)
print("Z-test p-value:", p_value_z)


Z-test statistic: -2.4226047226272107
Z-test p-value: 0.0154096835916544


In [12]:
sigma2_tilde = np.mean((data - mu0)**2)

LR_stat = n * np.log(sigma2_tilde / sigma2_hat)

p_value_LR = 1 - stats.chi2.cdf(LR_stat, df=1)

print("LR statistic:", LR_stat)
print("LR test p-value:", p_value_LR)


LR statistic: 5.784548778596572
LR test p-value: 0.016167647516077577


In [13]:
wald_stat = (mu_hat - mu0)**2 / (sigma2_hat / n)
p_value_wald = 1 - stats.chi2.cdf(wald_stat, df=1)

print("Wald statistic:", wald_stat)
print("Wald test p-value:", p_value_wald)


Wald statistic: 5.869013642095663
Wald test p-value: 0.015409683591654288


### part g: coefficient of variation

#### ii: MLE of coefficient of variation

In [14]:
cv_hat = np.sqrt(sigma2_hat) / mu_hat
print("MLE of coefficient of variation:", cv_hat)


MLE of coefficient of variation: 0.14637436146263474


#### iii:delta-method standard error

In [17]:
se_cv = np.sqrt(
    (sigma2_hat**2 / mu_hat**4 + sigma2_hat / (2 * mu_hat**2)) / n
)

ci_cv = (
    cv_hat - 1.96 * se_cv,
    cv_hat + 1.96 * se_cv
)

print("95% delta-method CI for CV:", ci_cv)


95% delta-method CI for CV: (np.float64(0.1317255563598098), np.float64(0.1610231665654597))
